In [ ]:
 %load_ext sql

In [ ]:
%sql sqlite:///ENS_RECORDS.db

In [ ]:
%%sql
/* Get the table names */
select name
from sqlite_master 
where type='table'

union all 

select name 
from sqlite_temp_master 
where type='table'

In [ ]:
%%sql
PRAGMA table_info(ens_names);

In [ ]:
%%sql
PRAGMA table_info(ens_records_resolved);

In [ ]:
%%sql
PRAGMA table_info(ens_records_resolved_errors);

In [ ]:
%%sql
SELECT *
from ens_names
LIMIT 5;

In [ ]:
%%sql
SELECT *
from ens_records_resolved
LIMIT 5;

In [ ]:
%%sql
SELECT *
from ens_records_resolved_errors
LIMIT 5;

## What key's are available for each ENS name

In [ ]:
%%sql
SELECT
    DISTINCT(key_value_table.key) as json_keys,
    COUNT(*) count_keys
FROM (
    SELECT j.*
    FROM ens_names t, json_each(ENS_RECORD_JSON) j
) as key_value_table
GROUP BY json_keys;

In [ ]:
%%sql
SELECT DISTINCT((json_extract(ENS_RECORD_JSON,'$.resolver.texts')))
FROM ens_names
LIMIT 10

In [ ]:
%%sql
SELECT COUNT(DISTINCT(sub_domain))
FROM ens_records_resolved
LIMIT 10

In [ ]:
%%sql
SELECT DISTINCT(sub_domain)
FROM ens_records_resolved
LIMIT 10;

In [ ]:
import sqlite3
import json
import pandas as pd
import pprint
from sqlalchemy import create_engine
import os 

In [ ]:
sqlite_path = './ENS_RECORDS.db'

In [ ]:
# [Get dictionary result from sqlite src](https://stackoverflow.com/a/3300514)
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d
con = sqlite3.connect(sqlite_path)
con.row_factory = dict_factory
cur = con.cursor()

In [ ]:
current_path = os.getcwd()
print(os.getcwd())

In [ ]:
cnx = create_engine("sqlite:///" + os.getcwd() + sqlite_path[1:])

In [ ]:
subdomain_list = list(cur.execute(""" 
SELECT DISTINCT(sub_domain)
FROM ens_records_resolved
"""))

In [ ]:
# print(subdomain_list)
actual_subdomain_list = []
for item in subdomain_list:
    actual_subdomain_list.append(item["sub_domain"])
pprint.pprint(actual_subdomain_list[:10])

In [ ]:
com_twitter = list(cur.execute(""" 
SELECT ens_name, sub_domain, ens_record_data
FROM ens_records_resolved
"""))
pprint.pprint(com_twitter[:10])

In [ ]:
pd.DataFrame(data=com_twitter)

In [ ]:
select_all_domains_and_records = pd.read_sql_query(""" 
SELECT ens_name, sub_domain, ens_record_data
FROM ens_records_resolved
""", con=cnx)

In [ ]:
display(select_all_domains_and_records)

In [ ]:
select_all_twitter_domains_and_records = pd.read_sql_query(""" 
SELECT ens_name, sub_domain, ens_record_data
FROM ens_records_resolved
WHERE sub_domain = "com.twitter"
""", con=cnx)

In [ ]:
display(select_all_twitter_domains_and_records)

In [ ]:
select_all_discord_domains_and_records = pd.read_sql_query(""" 
SELECT ens_name, sub_domain, ens_record_data
FROM ens_records_resolved
WHERE sub_domain = "com.discord"
""", con=cnx)

In [ ]:
select_all_discord_domains_and_records

In [ ]:
select_all_url_domains_and_records = pd.read_sql_query(""" 
SELECT ens_name, sub_domain, ens_record_data
FROM ens_records_resolved
WHERE sub_domain = "url"
""", con=cnx)

In [ ]:
display(select_all_url_domains_and_records)